In [43]:
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
# pd.set_option('display.max_columns', None)  # or 1000
pd.set_option('display.max_rows', None)  # or 1000 to get all the rows for our dataframe
# pd.set_option('display.max_colwidth', None)  # or 199


/kaggle/input/tabular-playground-series-sep-2021/train.csv
/kaggle/input/tabular-playground-series-sep-2021/test.csv
/kaggle/input/tabular-playground-series-sep-2021/sample_solution.csv


In [44]:
train_df = pd.read_csv("/kaggle/input/tabular-playground-series-sep-2021/train.csv")

In [45]:
test_df = pd.read_csv("/kaggle/input/tabular-playground-series-sep-2021/test.csv")

In [46]:
train_df.head(10)

,id,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f110,f111,f112,f113,f114,f115,f116,f117,f118,claim
0,0,0.108590,0.004314,-37.566,0.017364,0.289150,-10.251000,135.12,168900.0,3.992400e+14,...,-12.2280,1.7482,1.90960,-7.11570,4378.80,1.2096,8.613400e+14,140.1,1.017700,1
1,1,0.100900,0.299610,11822.000,0.276500,0.459700,-0.837330,1721.90,119810.0,3.874100e+15,...,-56.7580,4.1684,0.34808,4.14200,913.23,1.2464,7.575100e+15,1861.0,0.283590,0
2,2,0.178030,-0.006980,907.270,0.272140,0.459480,0.173270,2298.00,360650.0,1.224500e+13,...,-5.7688,1.2042,0.26290,8.13120,45119.00,1.1764,3.218100e+14,3838.2,0.406900,1
3,3,0.152360,0.007259,780.100,0.025179,0.519470,7.491400,112.51,259490.0,7.781400e+13,...,-34.8580,2.0694,0.79631,-16.33600,4952.40,1.1784,4.533000e+12,4889.1,0.514860,1
4,4,0.116230,0.502900,-109.150,0.297910,0.344900,-0.409320,2538.90,65332.0,1.907200e+15,...,-13.6410,1.5298,1.14640,-0.43124,3856.50,1.4830,-8.991300e+12,NaN,0.230490,1
5,5,0.101530,-0.002612,-1118.700,0.116300,0.318860,-0.478390,2372.80,-1808.0,5.818100e+15,...,-44.0820,3.5812,26.55900,-6.47220,44570.00,1.1776,2.997700e+14,5548.3,-0.033159,0
6,6,0.003073,0.359530,20913.000,0.003465,0.268060,-1.225100,1301.20,233180.0,1.771400e+15,...,-39.9220,1.4896,1.29230,33.63400,16201.00,1.2271,2.458900e+16,1012.9,0.647970,0
7,7,0.097340,0.245910,11775.000,0.614860,0.479370,-1.486500,3179.20,123940.0,1.652900e+15,...,-22.9950,1.5719,101.95000,1.51560,1117.50,1.2468,-6.971300e+13,1481.7,0.303170,1
8,8,-0.008948,0.338020,503.810,0.601520,0.261760,0.735380,1614.00,408030.0,7.684400e+14,...,-9.5142,2.2176,-1.23750,7.07130,1030.80,1.2040,5.290300e+15,1968.1,0.145930,0
9,9,0.126230,0.173960,1662.000,0.038081,0.000485,0.059909,296.07,-8035.3,4.526300e+15,...,-1.9684,3.9595,19.16900,-0.24498,838.39,1.1968,2.567000e+17,4914.9,0.519760,0


So, there are some 119 features and and a target variable claim. 
Let's first try to train the model without null values and will then impute the null values with some value. 

In [47]:
Summary = pd.DataFrame(train_df.dtypes, columns=['Dtype'])
Summary["max"] = train_df.max()
Summary["min"] = train_df.min()
Summary["Null"] = train_df.isnull().sum() # to get null values
Summary["First"] = train_df.iloc[0] # to get first value
Summary["Second"] = train_df.iloc[1] # to get second value
Summary


,Dtype,max,min,Null,First,Second
id,int64,9.579180e+05,0.000000e+00,0,0.000000e+00,1.000000e+00
f1,float64,4.151700e-01,-1.499100e-01,15247,1.085900e-01,1.009000e-01
f2,float64,5.189900e-01,-1.904400e-02,15190,4.313900e-03,2.996100e-01
f3,float64,3.954400e+04,-9.421700e+03,15491,-3.756600e+01,1.182200e+04
f4,float64,1.319900e+00,-8.212200e-02,15560,1.736400e-02,2.765000e-01
f5,float64,5.547500e-01,-6.989800e-03,15405,2.891500e-01,4.597000e-01
f6,float64,1.120200e+01,-1.279100e+01,15521,-1.025100e+01,-8.373300e-01
f7,float64,5.426600e+03,-2.248000e+02,15504,1.351200e+02,1.721900e+03
f8,float64,1.913700e+06,-2.984300e+04,15373,1.689000e+05,1.198100e+05
f9,float64,1.042400e+16,-1.153300e+15,15249,3.992400e+14,3.874100e+15


Almost all the rows have null values and the datatypes for all column is float except for claim, that is target variable, in that case it is int.

Let's drop the target variable and id from the training data.

In [48]:
# train_df = train_df.fillna(0)
train_df = train_df.fillna(train_df.mean())

In [49]:
print(len(train_df))

957919


In [50]:
test_df.head(10)

,id,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f109,f110,f111,f112,f113,f114,f115,f116,f117,f118
0,957919,0.165850,0.487050,1295.00,0.023100,0.31900,0.90188,573.29,3743.7,2.705700e+12,...,0.162530,-22.18900,2.0655,0.430880,-10.7410,81606.0,1.1940,1.980400e+14,2017.100,0.463570
1,957920,0.129650,0.373480,1763.00,0.728840,0.33247,-1.26310,875.55,554370.0,5.955700e+14,...,0.815280,-1.63420,1.5736,-1.071200,11.8320,90114.0,1.1507,4.388000e+16,6638.900,0.281250
2,957921,0.120190,0.445210,736.26,0.046150,0.29605,0.31665,2659.50,317140.0,3.977800e+14,...,0.818310,-32.78000,2.1364,-1.931200,-3.2804,37739.0,1.1548,1.718100e+14,5844.000,0.137970
3,957922,0.054008,0.395960,996.14,0.859340,0.36678,-0.17060,386.56,325680.0,-3.432200e+13,...,0.865590,-2.41620,1.5199,-0.011633,1.3840,26849.0,1.1490,2.138800e+17,6173.300,0.329100
4,957923,0.079947,-0.006919,10574.00,0.348450,0.45008,-1.84200,3027.00,428150.0,9.291500e+11,...,0.251900,-18.63000,3.7387,0.757080,-4.9405,50336.0,1.2488,2.151300e+17,2250.100,0.337960
5,957924,0.098700,0.289880,8499.10,0.558120,0.25080,0.11889,3099.90,307200.0,7.736000e+14,...,0.282030,-26.54600,1.3292,2.658000,6.2382,5051.9,1.3733,1.975900e+14,11405.000,0.843270
6,957925,NaN,0.495320,2274.40,0.744540,0.43810,3.56510,1534.20,102930.0,8.759700e+14,...,0.006470,-53.02200,1.6696,-1.613500,-9.0595,27646.0,1.3670,4.245900e+16,4488.000,1.616800
7,957926,0.131640,0.313620,2679.70,0.412410,0.48078,2.66990,1379.10,750760.0,4.893000e+15,...,0.055057,-23.64600,3.9702,1.335200,-8.7626,1883.9,1.2916,1.713500e+16,2386.400,0.060282
8,957927,0.174400,0.298570,-750.75,0.016088,0.00011,4.17340,667.93,634070.0,8.625400e+14,...,0.842840,-9.38620,1.5446,140.460000,-19.0160,50389.0,1.1299,1.514300e+16,4740.900,0.597500
9,957928,0.101710,0.123810,594.47,0.034546,0.00075,NaN,2490.20,550700.0,1.283400e+13,...,0.103770,0.14017,1.4772,1.158500,-2.9524,132270.0,NaN,1.146600e+16,-36.192,1.346200


We can see that the test_df also have 119 features. Target variable is not included in the dataset. 

Let's drop the id column from the test dataset as well and impute the missing values with zero.

In [51]:
# y.nunique.count()

In [52]:
test_df = test_df.drop("id", axis=1)

In [53]:
test_df = test_df.fillna(test_df.median())


In [54]:
y = train_df["claim"]
train_df = train_df.drop(["id"], axis=1)

In [55]:
import matplotlib.pyplot as plt
corr_mat = train_df.corr()

In [56]:
# corr_mat[abs(corr_mat) > 0.5]

cor_target = abs(corr_mat["claim"])
#Selecting highly correlated features
relevant_features = cor_target[cor_target<0.0009]
relevant_features

f18     0.000320
f26     0.000896
f49     0.000038
f58     0.000237
f66     0.000474
f72     0.000816
f74     0.000177
f75     0.000575
f76     0.000848
f84     0.000040
f101    0.000126
f115    0.000428
Name: claim, dtype: float64

In [57]:
feat_to_drop = relevant_features.index.to_list()
train_df = train_df.drop(feat_to_drop, axis = 1)
test_df = test_df.drop(feat_to_drop, axis = 1)

In [58]:
train_df = train_df.drop(["claim"], axis=1)

In [59]:
# test_df.isnull().sum()

In [60]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

X_train, X_test, y_train, y_test = train_test_split(train_df, y, random_state = 123, test_size = 0.2)

In [61]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [62]:
from xgboost import XGBRegressor

In [63]:
print(len(X_train))
print(len(X_test))
print(len(y_train))
print(len(y_test))

766335
191584
766335
191584


In [64]:
lr = LinearRegression()
lr.fit(X_train, y_train)


LinearRegression()

In [65]:
xgbr= XGBRegressor(
    n_estimators=3000,
    learning_rate=0.03,
    subsample=0.8,
    colsample_bytree=0.1,
    max_depth=3,
    booster='gbtree',
    reg_lambda=0.0008,
    reg_alpha=24,
    random_state=123
)
# xgbr = XGBRegressor(random_state = 123)
xgbr.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.03, max_delta_step=0, max_depth=3,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=3000, n_jobs=2, num_parallel_tree=1, random_state=123,
             reg_alpha=24, reg_lambda=0.0008, scale_pos_weight=1, subsample=0.8,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [66]:
y_pred_xgb = xgbr.predict(X_test)

In [67]:
y_pred = lr.predict(X_test)

In [68]:
from sklearn.metrics import roc_curve, auc

fpr, tpr, thresholds = roc_curve(y_test, y_pred)
print(auc(fpr, tpr))
# this is the score: 0.5254161343242256

0.5395700484683587


In [69]:
from sklearn.metrics import roc_curve, auc

fpr, tpr, thresholds = roc_curve(y_test, y_pred_xgb)
print(auc(fpr, tpr))
# this is the score: 0.776245437662909

0.7898822825912104


In [70]:
test_df = scaler.transform(test_df)

In [71]:
y_val = lr.predict(test_df)

In [72]:
y_val_xgbr = xgbr.predict(test_df)

In [73]:
sub = pd.read_csv("/kaggle/input/tabular-playground-series-sep-2021/sample_solution.csv")

In [74]:
y_val

array([0.49865942, 0.40981666, 0.4792779 , ..., 0.47748414, 0.43690072,
       0.47081279])

In [75]:
y_val_xgbr

array([0.3847936 , 0.28891012, 0.37998948, ..., 0.36942267, 0.31708097,
       0.38330048], dtype=float32)

In [76]:
sub["claim"] = y_val_xgbr

In [77]:
y_val_xgbr

array([0.3847936 , 0.28891012, 0.37998948, ..., 0.36942267, 0.31708097,
       0.38330048], dtype=float32)

In [78]:
y_res = (y_val_xgbr + y_val)/2

In [79]:
sub["claim"] = y_val_xgbr
sub.to_csv("submission.csv", index=False)
sub.head(10)

,id,claim
0,957919,0.384794
1,957920,0.288910
2,957921,0.379989
3,957922,0.331997
4,957923,0.347927
5,957924,0.491370
6,957925,0.409686
7,957926,0.360810
8,957927,0.347713
9,957928,0.335576
